# Summary

Although this was a large section which introduced several keywords and concepts, it has hopefully allowed you to see that T-SQL has some good and stable rules which we can follow in order to perform not only retrieval but also storage and manipulation of data.

You should also now be very comfortable with both table aliases and temporary tables that we have utilised throughout.

At the very start of this section we posed several challenges which seemed quite advanced at the time:

* Query One
 * Create a temporary table called #staffInfo to hold employeeID, firstName, lastName, a calculated fullName, jobTitle, salary, salesTotal
 * Populate with employee data for those with a firstName beginning with B or P
 * Employees must have had sales

* Query Two
 * Delete from #staffInfo where employee was born before 1970

* Query Three
 * Create an empty copy of the #staffInfo table called #staffAudit
 * Give all remaining staff a 10% raise in salary
 * Keep an audit of their previous salaries #staffAudit

However, now we are at the end of the section these should, hopefully, now seem quite trivial.

Let's work through them.

#### Query One

Firstly we shall create the temporary table in question, using the most applicable datatypes for each column:

In [ ]:
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);

This should suffice and will be used from now on in all three queries we are building.

The next requirement is to populate this with data for all employees who have firstNames beginning with B or P.

To start, we shall obtain all employees with firstNames beginning with B or P:

In [ ]:
select employeeID, firstName, lastName, jobTitleID, salary
from JupyterDatabase.hr.employee
where firstName like 'B%'
or firstName like 'P%';

Note that we only have the jobTitleID and not the jobTitle.  Therefore we need to JOIN this to the hr.jobTitle table:

In [ ]:
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
where e.firstName like 'B%'
or e.firstName like 'P%';

Finally, we need the salesTotal column populating with only those who have sales.  This means that we need an INNER JOIN (as opposed to LEFT or RIGHT) to the sales table (sales.salesOrderHeader):

In [ ]:
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;

Now that we have our dataset we can INSERT this into our temporary table to complete Query One:

In [ ]:
--------------------------------------------------------------------------------------------
-- Temp Table
--------------------------------------------------------------------------------------------
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);
--------------------------------------------------------------------------------------------

-- Populate with the data we have determined
insert into #staffInfo
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;

-- Check the results
select *
from #staffInfo;

#### Query Two

Now we have our data we need to DELETE any employee who was born before 1970.

As you will note, we do not have data of birth as a column in our table.  Therefore, in order to fulfil this query we are going to have to use our #staffInfo table and join back to hr.employee in order to obtain the data we need to DELETE.

Therefore we will be performing a DELETE via a JOIN, which we now know how to do.

Firstly we shall follow on from the above query in order to find those employees who were born before 1970 (ie. locate the records we are going to DELETE):

In [ ]:
--------------------------------------------------------------------------------------------
-- Temp Table
--------------------------------------------------------------------------------------------
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);

-- Populate
insert into #staffInfo
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;
--------------------------------------------------------------------------------------------

-- Join back to hr.employee to find those born before 1970
select s.*, e.dob
from #staffInfo as s
join JupyterDatabase.hr.employee as e
on s.employeeID = e.employeeID
where e.dob < '1970-01-01';

This looks correct and therefore tells us that we will be deleting two records.

Let's proceed with altering the SELECT into a DELETE (remove the SELECT line and replace with DELETE and the alias of the table we are deleting from):

In [ ]:
--------------------------------------------------------------------------------------------
-- Temp Table
--------------------------------------------------------------------------------------------
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);

-- Populate
insert into #staffInfo
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;
--------------------------------------------------------------------------------------------

-- Join back to hr.employee and DELETE those born before 1970
delete s
from #staffInfo as s
join JupyterDatabase.hr.employee as e
on s.employeeID = e.employeeID
where e.dob < '1970-01-01';

-- Check the results
select *
from #staffInfo;

This is now correct and so we have completed Query Two.

#### Query Three

The criteria for this query is that we start from where we left off in Query Two and then we give the remaining three staff members a 10% raise.

As we already have salary in our table this does not require any joins or other syntax but is a basic UPDATE.

However, we do need to create a copy of the #staffInfo table in order to keep an audit of our changes.

Although not explicitly mentioned, it is a good idea to place a datetime column in the Audit in order to track dates and times that changes were made.  Therefore we will take this upon ourselves to include.

Once done we will be utilising the OUTPUT and INTO keywords to fulfil the task as part of the UPDATE statement.

Firstly we shall use the script above but this time include the creation of our audit table along with only the UPDATE statment (no auditing).  This will look as follows:

In [ ]:
--------------------------------------------------------------------------------------------
-- Temp Tables
--------------------------------------------------------------------------------------------
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);

drop table if exists #staffAudit;

create table #staffAudit
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2),
    auditDateTime datetime default(current_timestamp)
);

-- Populate
insert into #staffInfo
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;
--------------------------------------------------------------------------------------------

-- Join back to hr.employee and DELETE those born before 1970
delete s
from #staffInfo as s
join JupyterDatabase.hr.employee as e
on s.employeeID = e.employeeID
where e.dob < '1970-01-01';

-- Perform the UPDATE on the table
update s
set s.salary = s.salary * 1.1
from #staffInfo as s;

-- Check the results
select *
from #staffInfo;

Now we have created our UPDATE statement we can add the OUTPUT and INTO keywords in order to log the audit trail and our report is complete:

In [ ]:
--------------------------------------------------------------------------------------------
-- Temp Tables
--------------------------------------------------------------------------------------------
drop table if exists #staffInfo;

create table #staffInfo
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2)
);

drop table if exists #staffAudit;

create table #staffAudit
(
    employeeID int,
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    jobTitle varchar(50),
    salary decimal(10, 2),
    salesTotal decimal(10, 2),
    auditDateTime datetime default(current_timestamp)
);

-- Populate
insert into #staffInfo
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
select e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary, sum(s.totalDue) as totalSales
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.sales.salesOrderHeader as s
on e.employeeID = s.salesPersonID
where e.firstName like 'B%'
or e.firstName like 'P%'
group by e.employeeID, e.firstName, e.lastName, t.jobTitle, e.salary;
--------------------------------------------------------------------------------------------

-- Join back to hr.employee and DELETE those born before 1970
delete s
from #staffInfo as s
join JupyterDatabase.hr.employee as e
on s.employeeID = e.employeeID
where e.dob < '1970-01-01';

-- Perform the UPDATE on the table
update s
set s.salary = s.salary * 1.1
output deleted.employeeID, deleted.firstName, deleted.lastName,
        deleted.jobTitle, deleted.salary, deleted.salesTotal
into #staffAudit
(
    employeeID, firstName, lastName, jobTitle, salary, salesTotal
)
from #staffInfo as s;

-- Check the results
select *
from #staffInfo;

-- Check the audit table
select *
from #staffAudit;